<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `NRPyPlusTOVID`: An Einstein Toolkit Thorn for Piecewise-Polytrope TOV neutron star initial data

## Author: Zach Etienne

[comment]: <> (Abstract: TODO)

**Module Status:** <font color='orange'><b> Partially Validated </b></font>

**Validation Notes:** NRPy+ TOV initial data generation module validated against [Josh Faber's TOV initial data solver](https://ccrg.rit.edu/~jfaber/BNSID/TOV/), as described in the [NRPy+ implementation notes of the TOV solution for piecewise-polytrope neutron stars](Tutorial-TOV-Piecewise_Polytrope_EOSs.ipynb).

### NRPy+ Source Code for this module: [TOV/TOV_Solver.py](../edit/TOV/TOV_Solver.py) [\[tutorial\]](Tutorial-Tutorial-ADM_Initial_Data-TOV.ipynb) Constructs numerical solution to TOV equations for neutron stars with piecewise polytrope equations of state

## Introduction:
In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up [TOV initial data](https://en.wikipedia.org/wiki/Tolman–Oppenheimer–Volkoff_equation) for an equilibrium neutron star. As documented in the [Piecewise Polytrope NRPy+ tutorial](Tutorial-TOV-Piecewise_Polytrope_EOSs.ipynb), piecewise-polytrope equations of state are supported, which closely approximate realistic nuclear equations of state appropriate for neutron star matter. In the [Tutorial-Tutorial-ADM_Initial_Data-TOV](Tutorial-Tutorial-ADM_Initial_Data-TOV.ipynb) tutorial module, we used NRPy+ to construct the SymPy expressions for these initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This module is organized as follows

1. [Step 1](#initializenrpy): Call on NRPy+ to generate the TOV solution given a piecewise-polytrope equation of state; output the data to a text file
1. [Step 2](#einstein): Interfacing with the Einstein Toolkit
    1. [Step 2.a](#einstein_c): Constructing the Einstein Toolkit C-code calling functions that include the C code kernels
    1. [Step 2.b](#einstein_ccl): CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure
    1. [Step 2.c](#einstein_list): Add the C code to the Einstein Toolkit compilation list
1. [Step 3](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Call on NRPy+ to generate the TOV solution given a piecewise-polytrope equation of state; output the data to a text file \[Back to [top](#toc)\]
$$\label{initializenrpy}$$



In [1]:
# Step 1: Import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

# Create directory for NRPyPlusTOVID thorn & subdirectories in case they don't exist.
outrootdir = "NRPyPlusTOVID/"
cmd.mkdir(os.path.join(outrootdir))
outdir = os.path.join(outrootdir,"src") # Main C code output directory
cmd.mkdir(outdir)

# Step 1.a: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1.b: NRPyPlusTOVID uses Cartesian coordinates, so
#           we specify the reference metric to be Cartesian here:
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

import TOV.Polytropic_EOSs as poly
# Set up the EOS parameters
eos = poly.set_up_EOS_parameters__Read_et_al_input_variables('APR4')


# Import our TOV solver, which supports both single
# and piecewise polytropic EOSs
import TOV.TOV_Solver as TOV

# Allocate memory to store rhoc, M and R_Schw
# We use a numpy linspace instead of declaring
# a list so that it is easier to manipulate the
# array later

# Set up the initial condition for the pressure by
# selecting a central baryon density
rhob_central = 2.0

# Set the initial value of M and R_Schw based
# on the initial central density
!rm -f rhob_P_cold_and_eps_cold.dat
import time
start = time.time()
Mass_TOV, R_TOV = TOV.TOV_Solver(eos,rho_baryon_central=rhob_central,verbose=True,return_M_and_RSchw=True)
print("TOV solution generated in: "+str(time.time()-start)+" s")

(1210, 1210, 1210, 1210, 1210, 1210)
Just generated a TOV star with R_Schw = 2.662450073694224e-01 , M = 8.795916086122126e-02 , M/R_Schw = 3.303692404611947e-01 .
TOV solution generated in: 5.73651099205 s


<a id='tov_interp'></a>

## Step 2.a: Interpolate the TOV data file as needed \[Back to [top](#toc)\]
$$\label{tov_interp}$$

The TOV data file just written stored $\left(r,\rho(r),P(r),M(r),e^{\nu(r)}\right)$, where $\rho(r)$ is the total mass-energy density (cf. $\rho_{\text{baryonic}}$).

### Step 2.a.i: Convert TOV solution quantities to `ADMBase` variables $\left\{\alpha,\beta^i,\gamma_{ij},K_{ij}\right\}$

The [TOV line element](https://en.wikipedia.org/wiki/Tolman%E2%80%93Oppenheimer%E2%80%93Volkoff_equation) in *Schwarzschild coordinates* is written (in the $-+++$ form):
$$
ds^2 = - c^2 e^\nu dt^2 + \left(1 - \frac{2GM}{rc^2}\right)^{-1} dr^2 + r^2 d\Omega^2.
$$

In *isotropic coordinates* with $G=c=1$ (i.e., the initial coordinate slicing and units we prefer to use), the ($-+++$ form) line element is written:
$$
ds^2 = - e^{\nu} dt^2 + e^{4\phi} \left(d\bar{r}^2 + \bar{r}^2 d\Omega^2\right),
$$
where $\phi$ here is the *conformal factor*.

The ADM 3+1 line element for this diagonal metric in isotropic spherical coordinates is given by:
$$
ds^2 = (-\alpha^2 + \beta_k \beta^k) dt^2 + \gamma_{\bar{r}\bar{r}} d\bar{r}^2 + \gamma_{\theta\theta} d\theta^2+ \gamma_{\phi\phi} d\phi^2,
$$

from which we can immediately read off the ADM quantities:
\begin{align}
\alpha &= e^{\nu(\bar{r})/2} \\
\beta^k &= 0 \\
\gamma_{\bar{r}\bar{r}} &= e^{4\phi}\\
\gamma_{\theta\theta} &= e^{4\phi} \bar{r}^2 \\
\gamma_{\phi\phi} &= e^{4\phi} \bar{r}^2 \sin^2 \theta \\
\end{align}

As this ETK module expects Cartesian coordinates, and the TOV solution above is in the spherical basis, we next perform the Jacobian transformations necessary to convert into the Cartesian basis:

All ADM tensors and vectors are in the Spherical coordinate basis $x^i_{\rm Sph} = (r,\theta,\phi)$, but we need them in the Cartesian coordinate basis $x^i_{\rm Cart}=$`(xx0,xx1,xx2)` set by the `"reference_metric::CoordSystem"` variable. Empirically speaking, it is far easier to write `(x(xx0,xx1,xx2),y(xx0,xx1, xx2),z(xx0,xx1,xx2))` than the inverse, so we will compute the Jacobian matrix

$$
{\rm Jac\_dUSph\_dDrfmUD[i][j]} = \frac{\partial x^i_{\rm Sph}}{\partial x^j_{\rm Cart}},
$$

via exact differentiation (courtesy SymPy), and the inverse Jacobian
$$
{\rm Jac\_dUrfm\_dDSphUD[i][j]} = \frac{\partial x^i_{\rm Cart}}{\partial x^j_{\rm Sph}},
$$

using NRPy+'s `generic_matrix_inverter3x3()` function. 

In terms of these, the transformation of ADM tensors from Spherical to `"reference_metric::CoordSystem==Cartesian"` coordinates may be written:

\begin{align}
\gamma^{\rm Cart}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm Sph}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm Sph}} \gamma^{\rm Sph}_{\ell m}
\end{align}

Since $\beta^i=K_{ij}=0$ in this case, and $\alpha$ is not a tensor, only the above Jacobian transformation need be performed:

In [2]:
thismodule   = __name__

IDalpha   = par.Cparameters("REAL", thismodule, "IDalpha", 1e300) # IDalpha must be set in C
IDbetaU   = ixp.zerorank1() # beta^i is zero
IDgammaDD = ixp.zerorank2()
for i in range(3):
    for j in range(i,3):
        IDgammaDD[i][j] = par.Cparameters("REAL", thismodule, "IDgammaDD"+str(i)+str(j), 1e300) # IDgammaDD must be set in C
        IDgammaDD[j][i] = IDgammaDD[i][j]
IDKDD     = ixp.zerorank2() # K_{ij} is zero

# Transform initial data to our coordinate system:
# First compute Jacobian and its inverse
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv()

# Declare as gridfunctions the final quantities we will output for the initial data
alpha   = gri.register_gridfunctions("EVOL","alpha")
betaU   = ixp.register_gridfunctions_for_single_rank1("EVOL","betaU")
gammaDD = ixp.register_gridfunctions_for_single_rank2("EVOL","gammaDD","sym01")
KDD     = ixp.register_gridfunctions_for_single_rank2("EVOL","KDD","sym01")

alpha = IDalpha # No Jacobian necessary!
betaU = IDbetaU # Because beta^i = 0
KDD   = IDKDD   # Because K_{ij} = 0

for i in range(3):
    for j in range(3):
        # Matrices are stored in row, column format, so (i,j) <-> (row,column)
        gammaDD[i][j] = 0
        for k in range(3):
            for l in range(3):
                gammaDD[i][j] += drrefmetric__dx_0UDmatrix[(k,i)]*drrefmetric__dx_0UDmatrix[(l,j)]*IDgammaDD[k][l]

# -={ Spacetime quantities: Generate C code from expressions and output to file }=-
ADMQuantities_to_print = [\
                          lhrh(lhs=gri.gfaccess("out_gfs","alpha"),rhs=alpha),\
                          lhrh(lhs=gri.gfaccess("out_gfs","betaU0"),rhs=betaU[0]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","betaU1"),rhs=betaU[1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","betaU2"),rhs=betaU[2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD00"),rhs=gammaDD[0][0]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD01"),rhs=gammaDD[0][1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD02"),rhs=gammaDD[0][2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD11"),rhs=gammaDD[1][1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD12"),rhs=gammaDD[1][2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","gammaDD22"),rhs=gammaDD[2][2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD00"),rhs=KDD[0][0]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD01"),rhs=KDD[0][1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD02"),rhs=KDD[0][2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD11"),rhs=KDD[1][1]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD12"),rhs=KDD[1][2]),\
                          lhrh(lhs=gri.gfaccess("out_gfs","KDD22"),rhs=KDD[2][2])
                          ]

with open(os.path.join(outdir,"ADMQuantities.h"),"w") as file:
    ADMQuantities_CcodeKernel = fin.FD_outputC("returnstring",ADMQuantities_to_print,
                                               params="outCverbose=False,includebraces=False,preindent=1")
    file.write("""
inline
void ADMQuantities(const CCTK_INT i0,const CCTK_INT i1,const CCTK_INT i2,

                   const CCTK_REAL IDalpha, 
                   const CCTK_REAL IDgammaDD00,const CCTK_REAL IDgammaDD01, const CCTK_REAL IDgammaDD02,
                   const CCTK_REAL IDgammaDD11,const CCTK_REAL IDgammaDD12, const CCTK_REAL IDgammaDD22,
                   
                   const CCTK_REAL *restrict xx0GF,const CCTK_REAL *restrict xx1GF,const CCTK_REAL *restrict xx2GF,
                   
                   CCTK_REAL *alphaGF,CCTK_REAL *betaU0GF,CCTK_REAL *betaU1GF,CCTK_REAL *betaU2GF,
                   CCTK_REAL *gammaDD00GF, CCTK_REAL *gammaDD01GF, CCTK_REAL *gammaDD02GF, 
                   CCTK_REAL *gammaDD11GF, CCTK_REAL *gammaDD12GF, CCTK_REAL *gammaDD22GF, 
                   CCTK_REAL *KDD00GF, CCTK_REAL *KDD01GF, CCTK_REAL *KDD02GF, 
                   CCTK_REAL *KDD11GF, CCTK_REAL *KDD12GF, CCTK_REAL *KDD22GF) {
    const CCTK_REAL xx0 = xx0GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)];
    const CCTK_REAL xx1 = xx1GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)];
    const CCTK_REAL xx2 = xx2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)];

"""+ADMQuantities_CcodeKernel+"""
}
    """)

### Step 2.a.ii: Convert TOV solution quantities to `HydroBase` variables $\left\{P,\rho_{\rm baryonic},\epsilon,v_{\rm (n)}^i\right\}$

The TOV solver outputs pressure $P$, the *total* energy density $\rho$, and the baryonic density $\rho_{\rm baryonic}$ as a function of the stellar radius (in isotropic coordinates by default). 

Then, the `HydroBase` quantities $\rho^{\rm HB}_{\rm baryonic}$, internal energy $\epsilon^{\rm HB}$, and pressure $P^{\rm HB}$ are given in terms of these variables via

\begin{align}
P^{\rm HB} &= P; \\
\rho^{\rm HB}_{\rm baryonic} &= \rho_{\rm baryonic}, \\
\rho &= \rho_{\rm baryonic} \left(1 + \epsilon_{\rm cold}\right) \\
\implies \epsilon_{\rm cold} &= \frac{\rho}{\rho_{\rm baryonic}} - 1\\
\epsilon^{\rm HB} &= \epsilon_{\rm cold}, \\
\end{align}
[the NRPy+ piecewise polytrope tutorial module](Tutorial-TOV-Piecewise_Polytrope_EOSs.ipynb#rhob_from_pcold). Note that $\rho_{\rm baryonic}$ will be floored to a nonzero atmosphere value, so that computing $\epsilon$ will never involve a division by zero.

The TOV star is motionless, with all spatial components of the 4-velocity $u^i=0$ and (as seen above) zero shift $\beta^i$. Thus the Valencia 3-velocity (i.e., the 3-velocity normal to the spatial slice) $v_{\rm (n)}^i$ is given by

$$
v_{\rm (n)}^{i,{\rm HB}} = 0
$$

In [3]:
IDValencia3velocityU = ixp.zerorank1() # Valencia 3-velocity is zero
IDPressure     = par.Cparameters("REAL", thismodule, "IDPressure", 1e300) # IDPressure must be set in C
IDrho_baryonic = par.Cparameters("REAL", thismodule, "IDrho_baryonic", 1e300) # IDrho_baryonic must be set in C
IDrho__total_energy_density = par.Cparameters("REAL", thismodule, "IDrho__total_energy_density", 1e300) # IDrho__total_energy_density must be set in C

# Declare as gridfunctions the final quantities we will output for the initial data
Valencia3velocityU = ixp.register_gridfunctions_for_single_rank1("EVOL","Valencia3velocityU")
Pressure, rho_baryonic, epsilon = gri.register_gridfunctions("EVOL",["Pressure", "rho_baryonic", "epsilon"])

Valencia3velocityU = IDValencia3velocityU # Because all components of Valencia3velocityU are *zero*
Pressure     = IDPressure
rho_baryonic = IDrho_baryonic
epsilon      = IDrho__total_energy_density / IDrho_baryonic - sp.sympify(1)

# -={ Spacetime quantities: Generate C code from expressions and output to file }=-
HydroQuantities_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","Pressure"),rhs=Pressure),\
                            lhrh(lhs=gri.gfaccess("out_gfs","rho_baryonic"),rhs=rho_baryonic),\
                            lhrh(lhs=gri.gfaccess("out_gfs","epsilon"),rhs=epsilon),\
                            lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU0"),rhs=Valencia3velocityU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU1"),rhs=Valencia3velocityU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","Valencia3velocityU2"),rhs=Valencia3velocityU[2])
                           ]

with open(os.path.join(outdir,"HydroQuantities.h"),"w") as file:
    HydroQuantities_CcodeKernel = fin.FD_outputC("returnstring",HydroQuantities_to_print,
                                               params="outCverbose=False,includebraces=False,preindent=2")
    file.write("""
inline
void HydroQuantities(const CCTK_INT i0,const CCTK_INT i1,const CCTK_INT i2,

                   const CCTK_REAL IDPressure, const CCTK_REAL IDrho_baryonic, 
                   const CCTK_REAL IDrho__total_energy_density,
                   
                   CCTK_REAL *PressureGF,CCTK_REAL *rho_baryonicGF,
                   CCTK_REAL *rho__total_energy_densityGF,
                   CCTK_REAL *Valencia3velocityU0GF,
                   CCTK_REAL *Valencia3velocityU1GF,
                   CCTK_REAL *Valencia3velocityU2GF) {
    CCTK_DECLARE_PARAMETERS;
    if(rho__total_energy_densityGF <= 0 || rho_baryonicGF <= 0 || PressureGF <= 0) {
        rho_baryonicGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = rho_atmosphere;
        PressureGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]     = K0_eos*pow(rho_atmosphere,Gamma_atmosphere);
        epsilonGF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)]      = 0;
        Valencia3velocityU0GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
        Valencia3velocityU1GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
        Valencia3velocityU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
    } else {
"""+HydroQuantities_CcodeKernel+"""
    }
}
    """)

Main driver function:

* Looping over all gridpoints:
    * Read in `const CCTK_REAL rr = r[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)];`
    * **Given this radius call interpolation driver to get all the base TOV quantities**
    * **Convert TOV spacetime quantities to ADM quantities in *spherical* basis**
    * Call the Cartesian ADMBase converter
    * Call the HydroBase converter

<a id='einstein'></a>

# Step 2: Interfacing with the Einstein Toolkit \[Back to [top](#toc)\]
$$\label{einstein}$$


<a id='einstein_c'></a>

## Step 2.a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels \[Back to [top](#toc)\]
$$\label{einstein_c}$$

We will write another C file with the functions we need here.

In [4]:
make_code_defn_list = []
def append_to_make_code_defn_list(filename):
    if filename not in make_code_defn_list:
        make_code_defn_list.append(filename)
    return os.path.join(outdir,filename)

with open(append_to_make_code_defn_list("InitialData.c"), "w") as file:
    file.write("""
#include <math.h>
#include <stdio.h>
#include <stdbool.h>
#include <stdlib.h> // Needed for rand()

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"

// Alias for "vel" vector gridfunction:
#define velx (&vel[0*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])
#define vely (&vel[1*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])
#define velz (&vel[2*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])

void NRPyPlusTOVID(const cGH* restrict const cctkGH,const CCTK_INT *cctk_lsh,
                   const CCTK_INT i0,const CCTK_INT i1,const CCTK_INT i2,
                   const CCTK_REAL *xcoordGF,const CCTK_REAL *ycoordGF,const CCTK_REAL *zcoordGF,
                   CCTK_REAL *alphaGF,CCTK_REAL *betaU0GF,CCTK_REAL *betaU1GF,CCTK_REAL *betaU2GF,
                   CCTK_REAL *gammaDD00GF,CCTK_REAL *gammaDD01GF,CCTK_REAL *gammaDD02GF,CCTK_REAL *gammaDD11GF,CCTK_REAL *gammaDD12GF,CCTK_REAL *gammaDD22GF,
                   CCTK_REAL     *KDD00GF,CCTK_REAL     *KDD01GF,CCTK_REAL     *KDD02GF,CCTK_REAL     *KDD11GF,CCTK_REAL     *KDD12GF,CCTK_REAL     *KDD22GF)
{
  
  DECLARE_CCTK_PARAMETERS
  
#include "KerrSchild.h"
  
}

void FishboneMoncrief_FMdisk_GRHD_velocities(const cGH* restrict const cctkGH,const CCTK_INT *cctk_lsh,
                                             const CCTK_INT i0,const CCTK_INT i1,const CCTK_INT i2,
                                             const CCTK_REAL *xcoordGF,const CCTK_REAL *ycoordGF,const CCTK_REAL *zcoordGF,
                                             CCTK_REAL *Valencia3velocityU0GF, CCTK_REAL *Valencia3velocityU1GF, CCTK_REAL *Valencia3velocityU2GF)
{
  
  DECLARE_CCTK_PARAMETERS
  
#include "FMdisk_GRHD_velocities.h"
  
}

void FishboneMoncrief_ET_GRHD_initial(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  CCTK_VINFO("Fishbone-Moncrief Disk Initial data.");
  CCTK_VINFO("Using input parameters of\n a = %e,\n M = %e,\nr_in = %e,\nr_at_max_density = %e\nkappa = %e\ngamma = %e",a,M,r_in,r_at_max_density,kappa,gamma);

  // First compute maximum density
  CCTK_REAL rho_max;
  {
    CCTK_REAL hm1;
    CCTK_REAL xcoord = r_at_max_density;
    CCTK_REAL ycoord = 0.0;
    CCTK_REAL zcoord = 0.0;
    {
#include "FMdisk_GRHD_hm1.h"
    }
    rho_max = pow( hm1 * (gamma-1.0) / (kappa*gamma), 1.0/(gamma-1.0) );
  }

#pragma omp parallel for
  for(CCTK_INT k=0;k<cctk_lsh[2];k++) for(CCTK_INT j=0;j<cctk_lsh[1];j++) for(CCTK_INT i=0;i<cctk_lsh[0];i++) {
        CCTK_INT idx = CCTK_GFINDEX3D(cctkGH,i,j,k);

        CCTK_REAL xcoord = x[idx];
        CCTK_REAL ycoord = y[idx];
        CCTK_REAL zcoord = z[idx];
        CCTK_REAL rr = r[idx];

        FishboneMoncrief_KerrSchild(cctkGH,cctk_lsh,
                                    i,j,k,
                                    x,y,z,
                                    alp,betax,betay,betaz,
                                    gxx,gxy,gxz,gyy,gyz,gzz,
                                    kxx,kxy,kxz,kyy,kyz,kzz);
        
        CCTK_REAL hm1;
        bool set_to_atmosphere=false;
        if(rr > r_in) {
          {
#include "FMdisk_GRHD_hm1.h"
          }
          if(hm1 > 0) {
            rho[idx] = pow( hm1 * (gamma-1.0) / (kappa*gamma), 1.0/(gamma-1.0) ) / rho_max;
            press[idx] = kappa*pow(rho[idx], gamma);
            // P = (\Gamma - 1) rho epsilon
            eps[idx] = press[idx] / (rho[idx] * (gamma - 1.0));
            FishboneMoncrief_FMdisk_GRHD_velocities(cctkGH,cctk_lsh,
                                                    i,j,k,
                                                    x,y,z,
                                                    velx,vely,velz);
          } else {
            set_to_atmosphere=true;
          }
        } else {
          set_to_atmosphere=true;
        }
        // Outside the disk? Set to atmosphere all hydrodynamic variables!
        if(set_to_atmosphere) {
          // Choose an atmosphere such that 
          //   rho =       1e-5 * r^(-3/2), and
          //   P   = k rho^gamma
          // Add 1e-100 or 1e-300 to rr or rho to avoid divisions by zero.
          rho[idx] = 1e-5 * pow(rr + 1e-100,-3.0/2.0);
          press[idx] = kappa*pow(rho[idx], gamma);
          eps[idx] = press[idx] / ((rho[idx] + 1e-300) * (gamma - 1.0));
          w_lorentz[idx] = 1.0;
          velx[idx] = 0.0;
          vely[idx] = 0.0;
          velz[idx] = 0.0;
        }
      }

  CCTK_INT final_idx = CCTK_GFINDEX3D(cctkGH,cctk_lsh[0]-1,cctk_lsh[1]-1,cctk_lsh[2]-1);
  CCTK_VINFO("=====   OUTPUTS   =====");
  CCTK_VINFO("betai: %e %e %e \ngij: %e %e %e %e %e %e \nKij: %e %e %e %e %e %e\nalp: %e\n",betax[final_idx],betay[final_idx],betaz[final_idx],gxx[final_idx],gxy[final_idx],gxz[final_idx],gyy[final_idx],gyz[final_idx],gzz[final_idx],kxx[final_idx],kxy[final_idx],kxz[final_idx],kyy[final_idx],kyz[final_idx],kzz[final_idx],alp[final_idx]);
  CCTK_VINFO("rho: %.15e\nPressure: %.15e\nvx: %.15e\nvy: %.15e\nvz: %.15e",rho[final_idx],press[final_idx],velx[final_idx],vely[final_idx],velz[final_idx]);
}

void FishboneMoncrief_ET_GRHD_initial__perturb_pressure(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  for(CCTK_INT k=0;k<cctk_lsh[2];k++) for(CCTK_INT j=0;j<cctk_lsh[1];j++) for(CCTK_INT i=0;i<cctk_lsh[0];i++) {
        CCTK_INT idx = CCTK_GFINDEX3D(cctkGH,i,j,k);
        // Generate random number in range [0,1),
        // snippet courtesy http://daviddeley.com/random/crandom.htm
        CCTK_REAL random_number_between_0_and_1 = ( (double)rand() / ((double)(RAND_MAX)+(double)(1)) );

        CCTK_REAL random_number_between_min_and_max = random_min + (random_max - random_min)*random_number_between_0_and_1;
        press[idx] = press[idx]*(1.0 + random_number_between_min_and_max);
        // Add 1e-300 to rho to avoid division by zero when density is zero.
        eps[idx] = press[idx] / ((rho[idx] + 1e-300) * (gamma - 1.0));
      }
}



SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-4-edd354bf88e7>, lines 8-143)

<a id='einstein_ccl'></a>

## Step 2.b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure \[Back to [top](#toc)\]
$$\label{einstein_ccl}$$

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. `interface.ccl}`: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

In [ ]:
%%writefile $outrootdir/interface.ccl
implements: NRPyPlusTOVID
inherits: admbase grid hydrobase


2. `param.ccl`: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [ ]:
%%writefile $outrootdir/param.ccl
shares: grid
shares: ADMBase
USES CCTK_INT lapse_timelevels
USES CCTK_INT shift_timelevels
USES CCTK_INT metric_timelevels

USES KEYWORD metric_type

EXTENDS KEYWORD initial_data 
{
  "NRPyPlusTOVID" :: "Initial data from NRPyPlusTOVID solution"
}
EXTENDS KEYWORD initial_lapse
{
  "NRPyPlusTOVID" :: "Initial lapse from NRPyPlusTOVID solution"
}
EXTENDS KEYWORD initial_shift
{
  "NRPyPlusTOVID" :: "Initial shift from NRPyPlusTOVID solution"
}
EXTENDS KEYWORD initial_dtlapse
{
  "NRPyPlusTOVID" :: "Initial dtlapse from NRPyPlusTOVID solution"
}
EXTENDS KEYWORD initial_dtshift
{
  "NRPyPlusTOVID" :: "Initial dtshift from NRPyPlusTOVID solution"
}

shares: HydroBase
EXTENDS KEYWORD initial_hydro
{
  "NRPyPlusTOVID" :: "Initial GRHD data from NRPyPlusTOVID solution"
}

#["r_in","r_at_max_density","a","M"] A_b, kappa, gamma
restricted:
CCTK_REAL r_in "Fixes the inner edge of the disk"
{
 0.0:* :: "Must be positive"
} 6.0

restricted:
CCTK_REAL r_at_max_density "Radius at maximum disk density. Needs to be > r_in"
{
 0.0:* :: "Must be positive"
} 12.0

restricted:
CCTK_REAL a "The spin parameter of the black hole"
{
 -1.0:1.0 :: "Positive values, up to 1. Negative disallowed, as certain roots are chosen in the hydro fields setup. Check those before enabling negative spins!"
} 0.9375

restricted:
CCTK_REAL M "Kerr-Schild BH mass. Probably should always set M=1."
{
 0.0:* :: "Must be positive"
} 1.0

restricted:
CCTK_REAL A_b "Scaling factor for the vector potential"
{
 *:* :: ""
} 1.0

restricted:
CCTK_REAL kappa "Equation of state: P = kappa * rho^gamma"
{
 0.0:* :: "Positive values"
} 1.0e-3

restricted:
CCTK_REAL gamma "Equation of state: P = kappa * rho^gamma"
{
 0.0:* :: "Positive values"
} 1.3333333333333333333333333333

##################################
# PRESSURE PERTURBATION PARAMETERS
private:
CCTK_REAL random_min "Floor value of random perturbation to initial pressure, where perturbed pressure = pressure*(1.0 + (random_min + (random_max-random_min)*RAND[0,1)))"
{
  *:*      :: "Any value"
} -0.02

private:
CCTK_REAL random_max "Ceiling value of random perturbation to initial pressure, where perturbed pressure = pressure*(1.0 + (random_min + (random_max-random_min)*RAND[0,1)))"
{
  *:*      :: "Any value"
} 0.02



3. `schedule.ccl`: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [ ]:
%%writefile $outrootdir/schedule.ccl
STORAGE: ADMBase::metric[metric_timelevels], ADMBase::curv[metric_timelevels], ADMBase::lapse[lapse_timelevels], ADMBase::shift[shift_timelevels]

schedule FishboneMoncrief_ET_GRHD_initial IN HydroBase_Initial 
{
  LANG: C
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: admbase::alp(Everywhere)
  WRITES: admbase::betax(Everywhere)
  WRITES: admbase::betay(Everywhere)
  WRITES: admbase::betaz(Everywhere)
  WRITES: admbase::kxx(Everywhere)
  WRITES: admbase::kxy(Everywhere)
  WRITES: admbase::kxz(Everywhere)
  WRITES: admbase::kyy(Everywhere)
  WRITES: admbase::kyz(Everywhere)
  WRITES: admbase::kzz(Everywhere)
  WRITES: admbase::gxx(Everywhere)
  WRITES: admbase::gxy(Everywhere)
  WRITES: admbase::gxz(Everywhere)
  WRITES: admbase::gyy(Everywhere)
  WRITES: admbase::gyz(Everywhere)
  WRITES: admbase::gzz(Everywhere)
  WRITES: hydrobase::velx(Everywhere)
  WRITES: hydrobase::vely(Everywhere)
  WRITES: hydrobase::velz(Everywhere)
  WRITES: hydrobase::rho(Everywhere)
  WRITES: hydrobase::eps(Everywhere)
  WRITES: hydrobase::press(Everywhere)
} "Set up general relativistic hydrodynamic (GRHD) fields for Fishbone-Moncrief disk"

schedule FishboneMoncrief_ET_GRHD_initial__perturb_pressure IN CCTK_INITIAL AFTER Seed_Magnetic_Fields BEFORE IllinoisGRMHD_ID_Converter
{
    LANG: C
} "Add random perturbation to initial pressure, after seed magnetic fields have been set up (in case we'd like the seed magnetic fields to depend on the pristine pressures)"


<a id='einstein_list'></a>

## Step 2.c: Add the C code to the Einstein Toolkit compilation list \[Back to [top](#toc)\]
$$\label{einstein_list}$$

We will also need `make.code.defn`, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [ ]:
with open(os.path.join(outdir,"make.code.defn"), "w") as file:
    file.write("""
# Main make.code.defn file for thorn NRPyPlusTOVID

# Source files in this directory
SRCS =""")
    filestring = ""
    for i in range(len(make_code_defn_list)):
        filestring += "      "+make_code_defn_list[i]
        if i != len(make_code_defn_list)-1:
            filestring += " \\\n"
        else:
            filestring += "\n"
    file.write(filestring)

<a id='latex_pdf_output'></a>

# Step 3: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ETK_thorn-NRPyPlusTOVID.pdf](Tutorial-ETK_thorn-NRPyPlusTOVID.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ETK_thorn-NRPyPlusTOVID.ipynb
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-NRPyPlusTOVID.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-NRPyPlusTOVID.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-NRPyPlusTOVID.tex
!rm -f Tut*.out Tut*.aux Tut*.log